In [44]:
pip install quandl

This program predicts the stock prices for Tesla Motors by using machine learning models. 

In [45]:
# DEPENDENCIES
import quandl
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split

In [46]:
# Get the stock data
df = quandl.get("WIKI/TSLA")
# Print head of the dataframe
df.head()

,Open,High,Low,Close,Volume,Ex-Dividend,Split Ratio,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume
Date,,,,,,,,,,,,
2010-06-29,19.00,25.0000,17.54,23.89,18766300.0,0.0,1.0,19.00,25.0000,17.54,23.89,18766300.0
2010-06-30,25.79,30.4192,23.30,23.83,17187100.0,0.0,1.0,25.79,30.4192,23.30,23.83,17187100.0
2010-07-01,25.00,25.9200,20.27,21.96,8218800.0,0.0,1.0,25.00,25.9200,20.27,21.96,8218800.0
2010-07-02,23.00,23.1000,18.71,19.20,5139800.0,0.0,1.0,23.00,23.1000,18.71,19.20,5139800.0
2010-07-06,20.00,20.0000,15.83,16.11,6866900.0,0.0,1.0,20.00,20.0000,15.83,16.11,6866900.0


In [47]:
# Get the adjusted close price
df = df[['Adj. Close']]

# Look at the new data
df.head()

,Adj. Close
Date,
2010-06-29,23.89
2010-06-30,23.83
2010-07-01,21.96
2010-07-02,19.20
2010-07-06,16.11


In [48]:
# A variable for predictinging 'n' days out into the future 
forecast_out = 1 

# Create another column (the target or dependent variable) shifted in units up
df['Prediction'] = df[['Adj. Close']].shift(-1)
# Printing the new dataset
df.head()

,Adj. Close,Prediction
Date,,
2010-06-29,23.89,23.83
2010-06-30,23.83,21.96
2010-07-01,21.96,19.20
2010-07-02,19.20,16.11
2010-07-06,16.11,15.80


In [49]:
# A variable for predicting 'n' days out into the future
forecast_out = 1
# Create another column (the target or dependent variable) shifted 'n' units up
df['Prediction'] = df[['Adj. Close']].shift(-forecast_out)
# Print the new dataset
print(df.tail())

            Adj. Close  Prediction
Date                              
2018-03-21      316.53      309.10
2018-03-22      309.10      301.54
2018-03-23      301.54      304.18
2018-03-26      304.18      279.18
2018-03-27      279.18         NaN


In [50]:
### Create the independent data set (x) ###
# Convert the dataframe to a numpy array
x = np.array(df.drop(['Prediction'],1))
# Remove the last 'n' rows
x = x[:-forecast_out]
print(x)

[[ 23.89]
 [ 23.83]
 [ 21.96]
 ...
 [309.1 ]
 [301.54]
 [304.18]]


In [51]:
### Create the dependent data set (y) ###
# Convert the datafram to a numpy array
y = np.array(df['Prediction'])
# Get all of the y values except the last 'n' rows
y = y[:-forecast_out]
print(y)

[ 23.83  21.96  19.2  ... 301.54 304.18 279.18]


In [52]:
# Split the data into 80% training and 20% testing
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)

In [53]:
# Create and train the Support Vector Machine (Regressor)
svr_rbf = SVR(kernel='rbf', C=1e3, gamma=0.1)
svr_rbf.fit(x_train, y_train)

SVR(C=1000.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.1,
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [54]:
# Testing Model: Score returns the coefficient of determination R^2 of the 
# prediction. The best possible score is 1.0
svm_confidence = svr_rbf.score(x_test, y_test)
print("svm confidence: ", svm_confidence)

svm confidence:  0.9927918574840524


In [55]:
# Create and train the Linear Regression Model
lr = LinearRegression()
# Train the model
lr.fit(x_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [56]:
# Testing Model: Score returns the coefficient of determination R^2 of the 
# prediction. The best possible score is 1.0
lr_confidence = lr.score(x_test, y_test)
print("lr confidence: ", lr_confidence)

lr confidence:  0.9976131288508342


In [57]:
# Set the x_forecast equal to the last 30 rows of the original data set from Adj.
# Close column
x_forecast = np.array(df.drop(['Prediction'],1))[-forecast_out:]
print(x_forecast)

[[279.18]]


In [58]:
# Print the predictions for the next 'n' days
lr_prediction = lr.predict(x_forecast)
print(lr_prediction)

[279.50271613]


In [59]:
# Print the support vector regressor model 
svm_prediction = svr_rbf.predict(x_forecast)
print(svm_prediction)

[280.54813752]
